# Namu Wiki JSON DB file parsing

## Reading DB Dump File

In [3]:
import json
import pickle
from pprint import pprint

# filename = 'namuwiki_170327.json' 

# # Read file to memory, it takes some time.
# with open(filename) as data_file:    
#     data = json.load(data_file)

# with open('namuwiki.pickle','wb') as pickle_file:
#     pickle.dump(pickle_file,data)

In [4]:
with open('namuwiki.pickle') as f: data = pickle.load(f)

## Data Exploration

In [5]:
# data is list of articles
# Let's see how many articles in the database
print("number of articles:", len(data)) 

# Let's see the first article
print("The first article is:")
print(data[0])

('number of articles:', 931029)
The first article is:
{u'text': u'#redirect \ub290\ub08c\ud45c\n', u'namespace': u'0', u'contributors': [u'namubot', u'R:hoon12560'], u'title': u'!'}


In [6]:
x = data[201704]

In [7]:
x.keys()

[u'text', u'namespace', u'contributors', u'title']

In [8]:
ns = {e['namespace']:1 for e in data}
ns

{u'0': 1, u'1': 1, u'2': 1, u'3': 1, u'4': 1, u'6': 1}

In [9]:
ts = {e['title']:1 for e in data}

In [10]:
print(ts.keys()[2])

저승사자 고즈


In [11]:
for key,val in x.items():
    print('===== key =====')
    print(key)
    print('===== val =====')
    print(val)

===== key =====
text
===== val =====
|||||||||| [[만포선]] ||<|2>'''관리 주체''' : [[조선민주주의인민공화국]] 철도성[br]'''소속''' : 개천철도총국 ||
||<width=88px> [[순천역(평안남도)|순　천]] 방면[br][[운송역|운　송]] || {{{+1 ←}}} ||<width=60px> 전　천 || {{{+1 →}}} ||<width=88px> [[만포청년역|만포청년]] 방면[br][[화암역|화　암]] ||

{{{+1 前川驛 / Jŏnchŏn Station }}}

[[만포선]]의 철도역. [[자강도]] [[전천군]] 전천읍 소재.

전천군이 비록 북한 체제하에서 1949년에 만들어진 군이기는 하지만, 전천이라는 지명은 강계군 전천면이라는 이름으로 일제 강점기부터 내려오고 있었다. 이 전천면의 중심지를 전천군의 중심지로 둔 것. 지도를 보면 구 전천면이 동서로 길게 있는 주제에 현 전천읍은 구 전천면의 남동쪽 끄트머리에 위치해 있는 것을 알 수 있는데, 이것은 전천면 서쪽이 모조리 산지이기 때문.

고도 50km 정도에서 이 일대의 위성 사진을 보면 [[장자강]](將子江, 독로강이라고도 함)을 따라서 하얀 부분(=나무가 없는 부분, 즉 경작지 또는 민가)이 쭉 이어져 있는 것을 알 수 있다. 전천읍도 그 중에 있고.

군의 중심지이기 때문인지 민가 및 수요는 상당한 편이라고 할 수 있다. 다만 그만큼 산림의 파괴가 극심한 것이 문제. 다른 마을보다 훨씬 파괴된 면적이 넓다. 자강도의 산지에 있는 군답게 주로 목재를 가공해서 먹고 산다고 한다. [[강계]]까지는 북쪽으로 55km.

승강장은 2면 5선으로 규모가 좀 있는 편이다. 인입선이 [[운송역]] 방향으로 나 있는데, 목재를 수송하기 위한 목적으로 보인다.

[[순천역(평안남도)|순천역]] 기점 195.4km. [[점촌역]] 구내에 있는 철도거리표에는 203.1km로 나와 있지만, 결론부터 말해서 신뢰할 수 없다. 해당 지점에도 역이 

In [12]:
type(x['title'])

unicode

In [13]:
# this black list article does not contain natural language knowledge
black_list_title = [u'공지사항/차단 내역/통합본']

black_listed_count = 0

# Check some statistics of whole dataset
count_dict = {}
for article in data:
    if article['title'] in black_list_title:
        black_listed_count += 1
        continue # remove blacklist article
        
#     if(len(article['text']) > 10000 and len(article['text']) < 11000):
#         print(article)
#         break
        
    if count_dict.get(len(article['text'])) == None:
        count_dict[len(article['text'])] = 1
    else:
        count_dict[len(article['text'])] = count_dict[len(article['text'])] + 1        
    

print('black listed count',black_listed_count)
print("min text size:", min(count_dict.keys()))
print("max text size:", max(count_dict.keys()))


('black listed count', 1)
('min text size:', 0)
('max text size:', 426441)


In [14]:
max(count_dict.keys())

426441

In [15]:
len(count_dict.keys())

33328

In [16]:
MAX_ARTICLE_SIZE = max(count_dict.keys())

bucket_size = 1000
num_bucket = MAX_ARTICLE_SIZE // bucket_size + 1

print('num_bucket:', num_bucket)

bucket_counts = [0] * num_bucket
for key, value in count_dict.items():
    index = key // bucket_size
    bucket_counts[index] = bucket_counts[index] + value

print(bucket_counts)



('num_bucket:', 427)
[655452, 75394, 47663, 31185, 21594, 15765, 12035, 9436, 7664, 6338, 5177, 4409, 3882, 3295, 2843, 2515, 2256, 1952, 1791, 1559, 1440, 1326, 1190, 1012, 944, 836, 771, 761, 727, 575, 589, 529, 495, 424, 382, 399, 379, 321, 309, 288, 297, 281, 269, 225, 200, 182, 182, 177, 179, 147, 120, 156, 140, 119, 122, 97, 92, 100, 92, 97, 83, 98, 77, 72, 76, 66, 56, 51, 48, 51, 53, 44, 38, 40, 35, 32, 43, 38, 34, 32, 30, 31, 24, 25, 31, 19, 19, 13, 21, 25, 10, 17, 11, 18, 18, 14, 18, 11, 22, 12, 19, 11, 13, 10, 13, 8, 15, 8, 10, 15, 6, 11, 6, 13, 10, 12, 7, 7, 6, 8, 10, 10, 1, 4, 7, 4, 3, 6, 6, 6, 3, 5, 9, 1, 3, 5, 4, 1, 4, 3, 3, 4, 4, 1, 1, 2, 1, 1, 0, 0, 1, 4, 3, 3, 2, 0, 0, 0, 2, 4, 1, 2, 1, 0, 0, 0, 0, 1, 1, 2, 2, 2, 1, 0, 2, 0, 0, 1, 1, 4, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 2, 1, 1, 0, 1, 0, 1, 2, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Test parsing

In [17]:
# Article contains title, text, and other things
# Let's extract title and text from several articles
for i in range(3):
    print('===== title =====')
    print(data[i]['title'])
    print('===== text =====')
    print(data[i]['text'])
    print()

===== title =====
!
===== text =====
#redirect 느낌표

()
===== title =====
!!아앗!!
===== text =====
[[파일:3444050440.jpg]]
([[신 세계수의 미궁 2]]에서 뜬 !!아앗!!)
{{{+1 ！！ああっと！！ }}}

[[세계수의 미궁 시리즈]]에 전통으로 등장하는 대사. [[세계수의 미궁 2 제왕의 성배|2편 제왕의 성배]]부터 등장했으며, 훌륭한 [[사망 플래그]]의 예시이다.

세계수의 모험가들이 탐험하는 던전인 수해의 구석구석에는 채취/벌채/채굴 포인트가 있으며, 이를 위한 채집 스킬에 투자하면 제한된 채집 기회에 보다 큰 이득을 챙길 수 있다. 그러나 분배할 수 있는 스킬 포인트는 한정된 만큼 채집 스킬에 투자하는 만큼 전투 스킬 레벨은 낮아지게 된다.

 1. 채집용 캐릭터들로 이루어진 약한 파티(ex: [[레인저(세계수의 미궁 2)|레인저]] 5명)가 수해에 입장한다.
 1. 필드 전투를 회피하면서 채집 포인트에 도착해 열심히 아이템을 캐는 중에...
 1. '''!!아앗!!''' ~~라플레시아가 나타났다!~~
 '''이때 등장하는 것은 [[FOE(세계수의 미궁 시리즈)|FOE]]는 아닌 일단 필드 졸개지만, 훨씬 위 층에 등장하는 강력한 졸개이며 선 턴을 빼앗긴다!'''
 1. '''[[으앙 죽음|떡잎]]'''(hage)

작품마다 !!아앗!!의 세세한 모습은 다르다. 그 악랄함은 첫 등장한 작품이자 시리즈 중에서도 불친절하기로 정평이 난 2편이 절정이었는데, 그야말로 위의 !!아앗!! 시퀀스 그대로, 묻지도 따지지도 않고 채집할 때마다 일정 확률로 '''강제로''' 전투에 돌입해야 했다. 게다가 이럴 때 쓰라고 있는 레인저의 스킬 '위험감지(중간 확률로 적의 선제공격을 무효화)'는 작동하지 않았다!
[[세계수의 미궁 3 성해의 내방자|3편]], [[세계수의 미궁 4 전승의 거신|4편]]에는 숨통이 트이게도 채집 중 낮은 확률로 "좋은 아이템을 얻을 수 있을 것 같

## Preprocessing with RegEx

In [18]:
# Using regular expression, we can strip some grammar. Let's see how we can do it. 
import re
text = u"딴 사람도 아니고 프로팀 [[Counter Logic Gaming|CLG]] 소속 전 서포터 [[스티브 차우|차우스터]]가 남긴 말이다."
t1 = re.sub(r"\[\[([^\]|]*)\]\]", r'\1', text) # remove link
print(t1)
t2 = re.sub(r"\[\[(?:[^\]|]*\|)?([^\]|]+)\]\]", r'\1', text) # remove link
print(t2)

딴 사람도 아니고 프로팀 [[Counter Logic Gaming|CLG]] 소속 전 서포터 [[스티브 차우|차우스터]]가 남긴 말이다.
딴 사람도 아니고 프로팀 CLG 소속 전 서포터 차우스터가 남긴 말이다.


In [19]:
# We want only plain texts, so strip wiki grammer.
# Refer this link to know more about grammar. https://namu.wiki/w/%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4:%EB%AC%B8%EB%B2%95%20%EB%8F%84%EC%9B%80%EB%A7%90

# Use regular expression to capture some pattern

# see http://stackoverflow.com/questions/2718196/find-all-chinese-text-in-a-string-using-python-and-regex
chinese = re.compile(u'[⺀-⺙⺛-⻳⼀-⿕々〇〡-〩〸-〺〻㐀-䶵一-鿃豈-鶴侮-頻並-龎]', re.UNICODE)
japanese = re.compile(u'[\u3000-\u303f\u3040-\u309f\u30a0-\u30ff\uff00-\uff9f\u4e00-\u9faf\u3400-\u4dbf]', re.UNICODE)

# hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
# hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+')  # 위와 동일
# result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거


def strip(text):               
    text = re.sub(r"\{\{\{#\!html[^\}]*\}\}\}", '', text, flags=re.IGNORECASE|re.MULTILINE|re.DOTALL) # remove html
    text = re.sub(r"#redirect .*", '', text, flags=re.IGNORECASE) # remove redirect
    text = re.sub(r"\[youtube\(\w+\)\]", '', text, flags=re.IGNORECASE) # remove youtube
    text = re.sub(r"\[include\(.*", '', text, flags=re.IGNORECASE)
    text = re.sub(r"\[\[ ?:?분류:.*", '', text) # remove 분류
    text = re.sub(r"\[\[ ?:?파일:.*", '', text) # remove 파일
    text = re.sub(r"\[\[[^\]|]*\|([^\]|]+)\]\]", r'\1', text) # remove link
    text = re.sub(r"\[\[([^\]|]*)(#[^\]|]+)?\]\]", r'\1', text) # remove link
    text = re.sub(r"\[\*([^\]]*)\]", '', text) # remove 각주
    text = re.sub(r"\{\{\{([^\ }|]*) ([^\}|]*)\}\}\}", r'\2', text) # remove text color/size
    text = re.sub(r"\* 상위 문서:.*", '', text) # remove 상위문서
    text = re.sub(r"'''([^']*)'''", r'\1', text) # remove text bold
    text = re.sub(r"==([^=]*)==", '', text) # == sdfsd ==
    text = re.sub(r"~~([^~]*)~~", '', text) # remove strike-through
    text = re.sub(r"--([^-]*)--", '', text) # remove strike-through

    text = re.sub(r"\|\|(.*)\|\|", '', text) # remove table

    # XXX workaround
    #text = re.sub(r".*문서:.*", '', text)
    #text = re.sub(r".*분류:.*", '', text)


    text = chinese.sub('', text) # remove chinese
    text = japanese.sub('', text) # remove japanese

    return text

In [20]:
text = """
* 상위 문서: [[개그 콘서트/종영 코너]]
[[분류:개그 콘서트 코너]]
"""
print(strip(text))

In [21]:
for n,s in enumerate(data[2]['text'].encode('utf-8').split('\n')):
    print n,s

0 [include(틀:다른 뜻1, other1=말줄임표 등으로 사용하는 용어, rd1=(...))]
1 
2  * 상위 문서: [[개그 콘서트/종영 코너]]
3 
4 ||<table align=right>|| http://www.mftp.info/20150901/1443910706x-576894271.jpg?width=320 ||
5 |||| “……” ||
6 || 참여 프로그램 || [[개그 콘서트]] ||
7 || 시작 || [[2013년]] [[5월 26일]] ||
8 || 종료 || 2013년 [[10월 13일]] ||
9 || 출연진 || [[유민상#s-1|유민상]], [[김희원]], [[송필근]], 남궁경호 ||
10 || 유행어 || XX는 더 이상합니다[* 어색한 상황을 수습해보려는 변명이 더 안 좋은 결과를 낼 때 나오는 말.] ||
11 
12 --[[나는 킬러다|2년 뒤에는 사위가 장인을 죽이려한다]]--
13 
14 '''진짜로 코너 [[이름]]이 ([[말풍선]])“……”[* 큰 따옴표도 포함된다.](말풍선)이다'''. 언론에서는 이 코너를 언급할 때 '점점점' 또는 '점점점점점점'이라는 표현을 사용한다. 코너명의 유래는 흔히 쓰이는 [[...|말줄임표]]. 2013년 5월 26일부터 방송되었고 [[김희원]], 남궁경호, [[유민상#s-1|유민상]], [[송필근]]이 출연한다.
15 
16 남자친구(남궁경호)와 여자친구(김희원)의 아버지(유민상) 간의 어색함을 다룬 코미디. 코너 초창기에는 남자친구 역인 남궁경호의 비중이 높았고 송필근은 배달부나 이웃주민 등 조연 역할이었으나, 송필근이 남궁경호의 아버지 역할(즉 유민상과 사돈관계) 컨셉으로 고정되면서 남궁경호의 비중이 줄어들고[* 심지어 아예 등장하지 않는 회차도 간간히 있다.] 사실상 유민상과 송필근의 투톱 체제로 나가고 있다. 따라서 그 이후로는 주로 사돈 간의 어색함을 다루고 있다. 3인(혹은 4인)이 있다가 여자친구(혹은 경호-희원 커플)가 자리를 비우면 남은 두 사람은 어색해 

In [22]:
print(data[2]['text'])

[include(틀:다른 뜻1, other1=말줄임표 등으로 사용하는 용어, rd1=(...))]

 * 상위 문서: [[개그 콘서트/종영 코너]]

||<table align=right>|| http://www.mftp.info/20150901/1443910706x-576894271.jpg?width=320 ||
|||| “……” ||
|| 참여 프로그램 || [[개그 콘서트]] ||
|| 시작 || [[2013년]] [[5월 26일]] ||
|| 종료 || 2013년 [[10월 13일]] ||
|| 출연진 || [[유민상#s-1|유민상]], [[김희원]], [[송필근]], 남궁경호 ||
|| 유행어 || XX는 더 이상합니다[* 어색한 상황을 수습해보려는 변명이 더 안 좋은 결과를 낼 때 나오는 말.] ||

--[[나는 킬러다|2년 뒤에는 사위가 장인을 죽이려한다]]--

'''진짜로 코너 [[이름]]이 ([[말풍선]])“……”[* 큰 따옴표도 포함된다.](말풍선)이다'''. 언론에서는 이 코너를 언급할 때 '점점점' 또는 '점점점점점점'이라는 표현을 사용한다. 코너명의 유래는 흔히 쓰이는 [[...|말줄임표]]. 2013년 5월 26일부터 방송되었고 [[김희원]], 남궁경호, [[유민상#s-1|유민상]], [[송필근]]이 출연한다.

남자친구(남궁경호)와 여자친구(김희원)의 아버지(유민상) 간의 어색함을 다룬 코미디. 코너 초창기에는 남자친구 역인 남궁경호의 비중이 높았고 송필근은 배달부나 이웃주민 등 조연 역할이었으나, 송필근이 남궁경호의 아버지 역할(즉 유민상과 사돈관계) 컨셉으로 고정되면서 남궁경호의 비중이 줄어들고[* 심지어 아예 등장하지 않는 회차도 간간히 있다.] 사실상 유민상과 송필근의 투톱 체제로 나가고 있다. 따라서 그 이후로는 주로 사돈 간의 어색함을 다루고 있다. 3인(혹은 4인)이 있다가 여자친구(혹은 경호-희원 커플)가 자리를 비우면 남은 두 사람은 어색해 하면서 아무 말 없이 중간중간 몇 마디 말만 던지고 다시 조용한 분위기가 

In [23]:
print(strip(data[2]['text']))



 * 상위 문서: 개그 콘서트/종영 코너











진짜로 코너 이름이 (말풍선)“……”(말풍선)이다. 언론에서는 이 코너를 언급할 때 '점점점' 또는 '점점점점점점'이라는 표현을 사용한다. 코너명의 유래는 흔히 쓰이는 말줄임표. 2013년 5월 26일부터 방송되었고 김희원, 남궁경호, 유민상, 송필근이 출연한다.

남자친구(남궁경호)와 여자친구(김희원)의 아버지(유민상) 간의 어색함을 다룬 코미디. 코너 초창기에는 남자친구 역인 남궁경호의 비중이 높았고 송필근은 배달부나 이웃주민 등 조연 역할이었으나, 송필근이 남궁경호의 아버지 역할(즉 유민상과 사돈관계) 컨셉으로 고정되면서 남궁경호의 비중이 줄어들고 사실상 유민상과 송필근의 투톱 체제로 나가고 있다. 따라서 그 이후로는 주로 사돈 간의 어색함을 다루고 있다. 3인(혹은 4인)이 있다가 여자친구(혹은 경호-희원 커플)가 자리를 비우면 남은 두 사람은 어색해 하면서 아무 말 없이 중간중간 몇 마디 말만 던지고 다시 조용한 분위기가 되는 것이 주된 패턴이다. 자리마다 항상 큰 괘종시계가 있어서 아무 말 없을 때마다 째깍째깍 소리가 나는 것이 웃음 포인트. 유민상이 사돈어른(송필근)이 찾아오기 전에 집안에 있는 괘종시계를 미리 치워버린 에피소드도 있었는데, 사돈어른이 선물로 벽시계를 사오면서 어색한 상황에서 째깍째깍 소리가 나게 되었다.

중간에 제 3자(송필근)가 끼어드는데, 남자친구가 자리를 비우면 제 3자와 아버지 간에도 사이가 어색해진다. 단순히 대화가 없는 상황을 보여주기도 하고, 어떤 사람은 대화가 별로 없는 현대 사회를 보여주기도 한다고 해석하기도 한다.

김희원은 유일하게 어색해하지 않게 붙임성있게 말 잘 하지만, 대신 눈치가 없다. 그리고 그녀의 눈치없는 말로 주변은 더 어색해진다.

7화(2013년 7월 14일 방송분)에서는 사돈어른이 2PM 춤 추다가 입원해서 놀림당했는데 건너편에 미라처럼 깁스를 하고 있던 유인석이 한 손을 뻗어 심장박동을 시전했다.  그리고 아빠와 딸이 모두 박

In [24]:
# for i in range(2):
#     print(data[i]['title'])
#     # print(data[i]['text'])
#     print(strip(data[i]['text']))
#     print()

In [25]:
# Generate raw text corpus

MIN_TEXT_SIZE = 5000

count_processed = 0
count_skipped = 0

with open('input.txt', 'w') as f:
    for article in data:
        if len(article['text']) < MIN_TEXT_SIZE or len(article['text']) >= MAX_ARTICLE_SIZE:
            count_skipped += 1
            continue # skip too small, too large articles

        count_processed += 1
        text = strip(article['text'])
        #f.write("%s\n%s\n\n\n" % (article['title'].encode('utf-8'), text.encode('utf-8')))
        f.write(text.encode('utf-8'))
        f.write("\n")
        # print(article['title'])
        # print(article['text'])
        # print(text)
        

In [17]:
print('count_processed',count_processed)
print('count_skipped',count_skipped)

('count_processed', 99739)
('count_skipped', 831290)


In [18]:
del data

In [19]:
import re

def remove_non_words(line):
    re_flags = re.IGNORECASE # re.UNICODE | re.IGNORECASE
    # 영문자,숫자,일부특수문자 제외 모든 문자
    # 숫자사이에 있는 `.' 를 제외한 `.'
    re_nonwords = r'([^A-Za-z0-9가-힣%. ]|[.,](?!\d))+'
    line = re.sub(re_nonwords, ' ', line, flags=re_flags)
    line = re.sub(r'\s+', ' ', line, flags=re_flags) # collapse repetitive spaces
    line = line.strip() # remove leading/training spaces
    return line


In [20]:
from scan_count_words import parse_words

In [21]:
import sqlite3

In [22]:
with sqlite3.connect('wordscount.db') as db:
    with open('text17.txt','w') as output:
        parse_words('input.txt',output,db)

100%|██████████| 14074786/14074786 [4:09:48<00:00, 939.07it/s]   
close
하 13642345
이 11623000
있 5759558
되 3968537
것 2821834
수 2276173
들 2203254
보 1579732
않 1542032
없 1512106
적 1490806
년 1442500
지 1334300
주 1330907
그 1141599
말 1055286
때문 1036804
때 1007790
아니 910671
등 885430
중 883856
가 837969
나오 818166
경우 805228
받 793236
1 768671
오 745978
크 726172
2 697839
보이 686578
같 678108
자신 652138
내 645126
사람 634242
사용 601564
이후 597472
많 595818
정도 594158
번 581530
쓰 579770
대하 575554
위하 559966
후 546128
일 540972
나 540472
3 513411
시 509227
만들 508302
좋 506614
가지 495958
공격 483446
위 483438
전 482200
알 477444
버리 469288
못하 465338
등장 455628
한 453162
명 444426
편 442736
안 442006
개 430584
사실 421778
게임 413804
타 412866
대 405904
하나 386080
리 385305
점 376784
4 374438
시작 367586
문제 356650
캐릭터 354784
모습 353276
만 348889
두 338262
높 337676
생각 334920
이상 329674
시간 326366
그렇 321398
. 317832
분류 315576
이름 314594
화 311828
따르 309658
자 305476
뒤 303064
더 298618
팀 298308
성 297504
능력 295786
살 295736
추가 293058
상대 292184
5 287585
상황 28647

In [1]:
!head -10 text17.txt

항성 대하 베텔게우스 항목 참조 
틀 루비 글자 루비 
당신 나태 
틀 루비 글자 루비 틀 루비 글자 루비 
뇌 떨리 아아 아아 아아 아아 아아 
목차 
개요 
라이트노벨 제 시작 세계 생활 등장인물 성우 마츠오카 시츠 이 
마녀 교 죄 주교 나태 담당 나이 세 키 몸무게 내외 최초 일 때 심 녹색 머리 단발 같 길이 자르 벌레 감정 없 눈 가지 깡마르 인물 고개 기울이 허리 굽히 자세 말 것 좋아하 스스로 육체 자학 것 즐기 완전 변 절 
이름 유래 오리온자리 알파 이 베텔게우스 베텔게우스 어원 이 아랍어 야드 알 자 우자 가운데 있 것 오리온 의 팔 늘 그 권능 관련 있 


In [30]:
# import sys
# reload(sys.modules[read_words_db.__module__])

In [31]:
from scan_txt_to_npy import read_words_db, dump_words, scan_txt_to_npy

In [32]:
words, words_index = read_words_db('wordscount.db',50000)

In [33]:
dump_words(words,'vocab_50k.tsv')

In [ ]:
num_words = scan_txt_to_npy('text17.txt', words_index, 'text17.npy')
num_words